# Build template using Advanced Normalization Tools (ANTs)
Specifically, use the scripts from the [optimized_antsMultivariateTemplateConstruction](https://github.com/CoBrALab/optimized_antsMultivariateTemplateConstruction) repository by CoBrA Lab
## Preparatory steps (need to be done before running this notebook)
1. __ANTs installation:__ Complile ANTs from source (on linux or Mac OS) following these [instructions](https://github.com/ANTsX/ANTs/wiki/Compiling-ANTs-on-Linux-and-Mac-OS)
2. __ANTs post-installation:__
   1. Complete the post-installation steps from above instructions to create `$ANTSPATH` and add it to your PATH.
   2. Copy all the scripts from the `Scripts` folder in the [ANTs repository](https://github.com/ANTsX/ANTs) to `$ANTSPATH`. Make sure that the scripts are executable.
3. Clone the [optimized_antsMultivariateTemplateConstruction](https://github.com/CoBrALab/optimized_antsMultivariateTemplateConstruction) and the [minc-toolkits-extras](https://github.com/CoBrALab/minc-toolkit-extras) repositories.
5. Copy all scripts from the cloned `optimized_antsMultivariateTemplateConstruction` as well as 2 scripts from the cloned `minc-toolkit-extras` to a _bin_ folder that's included in your PATH (could also be `$ANTSPATH`). Make sure that the scripts are executable.
6. Make sure to install [qbatch](https://github.com/pipitone/qbatch) via `pip install qbatch` and gnu-paraller via `sudo apt-get install parallel`

## Import libraries

In [1]:
import subprocess
import pathlib as pl

## Set paths

In [2]:
# Path to data (assert that folders exist)
data_folder = pl.Path('/home/nsirmpilatze/Data/bird-templates/BlackCap')
assert data_folder.is_dir(), f'{data_folder} is not a valid directory'

# Path to nifti images (Created by `Tiff to Nifti.ipynb` module)
data_nifti = data_folder / 'brains_nifti' # Destination for nifti files
assert data_nifti.exists(), f'{data_nifti} is not a valid directory'

# Folder where the template will be saved (create if it doesn't exist)
template_folder = data_folder / 'template_v1'
template_folder.mkdir(exist_ok=True)

## Template construction
Generate list of nifti images to be used for template construction and put the paths in a text file

In [3]:
with open( template_folder / 'inputs.txt', 'w' ) as f:
    nifti_paths = [str(p) for p in data_nifti.glob('*.nii')]
    for nifti_path in nifti_paths:
        print(nifti_path)
        f.write(nifti_path + '\n')

/home/nsirmpilatze/Data/bird-templates/BlackCap/brains_nifti/SW_BC38_ch03_green_050_micron.nii
/home/nsirmpilatze/Data/bird-templates/BlackCap/brains_nifti/SW_BC58_ch03_green_050_micron.nii
/home/nsirmpilatze/Data/bird-templates/BlackCap/brains_nifti/SW_BC73_ch03_green_050_micron.nii
/home/nsirmpilatze/Data/bird-templates/BlackCap/brains_nifti/SW_BC74_ch03_green_050_micron.nii


Select the image to be used as the initial registration target

In [4]:
# Choose the animal that was used for annotations
initial_target_id = 'SW_BC74'
initial_target_path = [nii for nii in nifti_paths if initial_target_id in nii][0]
print(initial_target_path)

/home/nsirmpilatze/Data/bird-templates/BlackCap/brains_nifti/SW_BC74_ch03_green_050_micron.nii


Call the `modelbuild.sh` script from the `optimized_antsMultivariateTemplateConstruction` repository

In [13]:
# script to call
script = 'bash modelbuild.sh'
# Path to inputs.txt
inputs = template_folder / 'inputs.txt'
# keyword arguments to pass to script
kw_args = {'output-dir': template_folder,
          'starting-target': initial_target_path,
          'stages': 'rigid,similarity,affine'}
cmd_kw_args = ' '.join([f'--{k} {v}' for k, v in kw_args.items()])
# flags to pass to script
flags = ['float']
cmd_flags = ' '.join([f'--{flag}' for flag in flags])
# build command string
cmd = f'{script} {cmd_kw_args} {cmd_flags} {inputs}'
print(cmd)
# Call it with subprocess
# subprocess(cmd, shell=True)


bash modelbuild.sh --output-dir /home/nsirmpilatze/Data/bird-templates/BlackCap/template_v1 --starting-target /home/nsirmpilatze/Data/bird-templates/BlackCap/brains_nifti/SW_BC74_ch03_green_050_micron.nii --stages rigid,similarity,affine --float /home/nsirmpilatze/Data/bird-templates/BlackCap/template_v1/inputs.txt
